In [1]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

## 生成context caching

In [4]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 读取文本文件内容
with open('prompts/DataFountain/answer.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# 创建缓存的请求数据
data = {
    "model": "moonshot-v1",
    "messages": [
        {
            "role": "system",
            "content": file_content
        }
    ],
    "name": "example_cache",
    "ttl": 3600  # 缓存有效期，单位为秒
}

# 发送POST请求创建缓存
response = requests.post(
    url="https://api.moonshot.cn/v1/caching",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=data
)

# 打印响应内容
print(json.loads(response.text))

{'id': 'cache-ey5p3ifxfuei11gkxpii', 'object': 'cache_object', 'model': 'moonshot-v1', 'messages': [{'role': 'system', 'content': '任务描述:\n一、输入格式：\n你将接收到一个包含以下四个字段的 JSON 数据：\n\ninstruction：问题的描述，明确了需要回答的内容。\ninput：与问题相关的参考材料，回答需基于其中的信息。\noutput：标准答案，用于作为高质量回答的参考。\nanswer：模型生成的回答，需要你进行评估。\n\n二、评估要求：\n请从以下几个方面对 answer 的质量进行评估，并为每个方面给出一个 0-100 的百分制评分，同时简要说明评分依据：\n\n1. 问题理解（Question Understanding）\nanswer 是否准确理解了 instruction 中的问题，并尝试回答所有要求的部分？\n是否遗漏或误解了问题的核心内容？\n\n2. 内容一致性（Content Consistency）\nanswer 是否基于 input 提供的信息作出回答？\n是否存在虚构信息（即未在 input 中提及却出现在 answer 中的内容）？\n\n3. 信息覆盖性（Coverage of Information）\nanswer 是否全面覆盖了 instruction 提出的问题，包括所有需要明确的方面？\n是否遗漏了 instruction 中提到的关键要素？\n\n4. 逻辑性和合理性（Logical Coherence and Reasonableness）\nanswer 是否逻辑自洽，符合常识和问题的场景？\n是否包含多余或不合理的推测？\n\n5. 语言表达（Clarity and Conciseness）\nanswer 是否语言清晰、简洁且易于理解？\n是否存在冗余、重复或表述不当的部分？\n\n6. 与标准答案的对比（Comparison to Standard Output）\nanswer 与标准 output 相比差距如何？\n是否包含标准 output 的核心要点？\n\n三、输出格式：\n你需要以 JSON 格式返回以下内容：\n\n```json\n{\n    

In [9]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 缓存ID，假设从创建缓存的响应中获取
cache_id = "cache-ey5p3ifxfuei11gkxpii"

# 发送DELETE请求删除缓存
response = requests.delete(
    url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 打印响应状态码
print(response.status_code)

# 如果需要，也可以打印响应内容
print(response.text)

200
{"deleted":true,"id":"cache-ey5p3ifxfuei11gkxpii","object":"cache_object.deleted"}


In [5]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"


# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']

## 对生成answer的质量进行评判

In [36]:
# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

In [37]:
# 主流程
file_path = 'prompts/DataFountain/answer.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

with open('ft_data/test_output_v2.json', 'r', encoding='utf-8') as file:
    answer = json.load(file)

judgement = []
for idx, value in tqdm(enumerate(answer), total=len(answer), desc="Processing"):
    context = answer[idx]

    # 使用缓存内容并添加问题
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement.append({
        "id": idx,
        "judgement": response
    })

with open('ft_data/judgement_v1.json', 'w', encoding='utf-8') as f:
    json.dump(judgement, f, indent=4, ensure_ascii=False)

Fileprompts/DataFountain/answer.txt success read
cache id:cache-ey5pxyxxfuei11gkxy4i
cache cuccess
缓存已存在


Processing: 100%|██████████| 499/499 [1:23:12<00:00, 10.00s/it]


In [40]:
print(judgement[0])

{'id': 0, 'judgement': '{\n    "scores": {\n        "question_understanding": 0,\n        "content_consistency": 0,\n        "coverage_of_information": 0,\n        "logical_coherence_and_reasonableness": 0,\n        "clarity_and_conciseness": 0,\n        "comparison_to_standard_output": 0\n    },\n    "explanations": {\n        "question_understanding": "answer 没有准确理解 instruction 中的问题，而是错误地将观测司的职责理解为监控和记录天体的位置、速度、距离和其他物理参数，这与问题要求回答的气象观测资料收集、传输和处理等职责完全不符。",\n        "content_consistency": "answer 中提到的内容与 input 提供的信息完全不一致，虚构了观测司监控和记录天体的职责，而 input 中明确指出观测司负责气象观测资料的收集、传输和处理等任务。",\n        "coverage_of_information": "answer 没有覆盖 instruction 提出的问题中的任何关键要素，而是提供了与问题无关的信息。",\n        "logical_coherence_and_reasonableness": "answer 中的内容逻辑上不自洽，与问题的场景完全不符，包含了不合理的推测。",\n        "clarity_and_conciseness": "虽然 answer 的语言表达清晰，但由于内容与问题完全不符，因此无法给出正面评价。",\n        "comparison_to_standard_output": "answer 与标准 output 相比差距极大，没有包含标准 output 的任何核心要点，完全偏离了问题的要求。"\n    }\n}'}
